In [4]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import numpy as np
import os
import sys
import warnings
warnings.filterwarnings('ignore')

In [11]:
lastyear = datetime.datetime.now()-datetime.timedelta(days=365)

begin_date = lastyear.strftime("%Y-%m-%d")
end_date = lastyear.strftime("%Y-%m-%d")

location_name = 'Abu Dhabi, AE' #location as input
mytoken = 'KvcVrPXahBeAZuNjikWrIWtbrnbosHyA'

url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations'
token = {'token': mytoken}   
r_loc = requests.get(url, headers = token)
df_loc = pd.DataFrame.from_dict(r_loc.json()['results'])
df_loc

locationid = df_loc[df_loc['name']==location_name]['id'][0]
print(locationid)

CITY:AE000001


In [12]:
def get_weather(locationid, begin_date, end_date, mytoken):
    token = {'token': mytoken}
    params = 'datasetid=GHCND'+'&locationid='+str(locationid)+'&startdate='+str(begin_date)+'&enddate='+str(end_date)+'&limit=1000'+'&units=standard'                  
    base_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'
    
    r = requests.get(base_url, params = params, headers=token,verify=False)
    print("Request status code: "+str(r.status_code))
    try:
        df = pd.DataFrame.from_dict(r.json()['results'])
        print("Successfully retrieved "+str(len(df['station'].unique()))+" stations")
        dates = pd.to_datetime(df['date'])
        print("Last date retrieved: "+str(dates.iloc[-1]))

        if df.count().max()==1000:
            print('Maximum data limit was reached (limit = 1000)')     
        return df
    except:
        print("Error converting weather data to dataframe.")

In [13]:
df_weather = get_weather(locationid, begin_date, end_date, mytoken)
df_weather

Request status code: 200
Successfully retrieved 1 stations
Last date retrieved: 2020-04-13 00:00:00


,date,datatype,station,attributes,value
0,2020-04-13T00:00:00,TAVG,GHCND:AEM00041217,"H,,S,",86.0


In [17]:
def get_station_info(locationid, mytoken):
    token = {'token': mytoken}   
    stations = 'locationid='+str(locationid)+'&'+'&'+'units=standard'+'&'+'limit=1000'
    base_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/stations'
    r = requests.get(base_url, headers = token, params=stations)
    print("Request status code: "+str(r.status_code))

    try:
        df = pd.DataFrame.from_dict(r.json()['results'])
        print("Successfully retrieved "+str(len(df['id'].unique()))+" stations")      
        if df.count().max() >= 1000:
            print('Maximum data limit was reached (limit = 1000)')
 
        return df
    except:
        print("Error converting station data to dataframe")

In [18]:
df_stations = get_station_info(locationid,mytoken)
df_stations

Request status code: 200
Successfully retrieved 1 stations


,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,26.8,1983-01-01,2021-04-09,24.433,"ABU DHABI INTERNATIONAL, AE",0.9976,GHCND:AEM00041217,METERS,54.651


In [19]:
df = df_weather.merge(df_stations, left_on = 'station', right_on = 'id', how='inner')
df.drop(['id','mindate','maxdate'], inplace=True, axis=1)
df

,date,datatype,station,attributes,value,elevation,latitude,name,datacoverage,elevationUnit,longitude
0,2020-04-13T00:00:00,TAVG,GHCND:AEM00041217,"H,,S,",86.0,26.8,24.433,"ABU DHABI INTERNATIONAL, AE",0.9976,METERS,54.651


In [ ]:
df.to_csv('weather_'+str(begin_date)+'_noaa.csv', encoding='utf-8', index=False)
# save csv file